In [82]:
#Necessary imports and constants.
from py2cytoscape.data.cyrest_client import CyRestClient
from py2cytoscape.data.cynetwork import CyNetwork
from py2cytoscape.data.cynetwork import BASE_URL_NETWORK
from py2cytoscape.data.style_client import StyleClient
import requests, json, time

cy = CyRestClient()

REST_ENDPOINT = 'http://localhost:1234'

In [83]:
#Query NDEx
ndex_message_body = {
    "serverUrl": "http://ndexbio.org/v2",
    "uuid": "583e62e1-1e5f-11e8-b939-0ac135e8bacf"
}

ndex_response = requests.post(REST_ENDPOINT + '/cyndex2/v1/networks', data=json.dumps(ndex_message_body), headers={'Content-Type': 'application/json'})
ndex_response_data=ndex_response.json()['data']

In [84]:
#Get the sub-network so we can perform operations on it. This requires a connection to the collections framework, as a well as direct instantiation of CyNetwork. This is not generally required, but necessary if we're using SUIDs directly.
collections_response = requests.get(REST_ENDPOINT + '/v1/collections/' + str(ndex_response_data['suid'])+'/subnetworks', headers={'Content-Type': 'application/json'})

subnetwork_suid = str(collections_response.json()[0])
print(subnetwork_suid)

subnetwork = CyNetwork(suid=subnetwork_suid,session=cy.session,url=BASE_URL_NETWORK)

#This is an annoying but necessary step... CyNDEx loads a network and returns, but Cytoscape may not have created a view for it. So, we check until the views endpoint returns something that isn't a 404
view_response = requests.get(REST_ENDPOINT + '/v1/networks/' + subnetwork_suid + '/views').status_code
retries = 10
while (view_response == 404 and retries > 0):
    view_response = requests.get(REST_ENDPOINT + '/v1/networks/' + subnetwork_suid + '/views').status_code
    time.sleep(5)
    retries = retries - 1

1716


In [85]:
#Run a layout algorithm
layout_name = 'kamada-kawai'
cy.layout.apply(name=layout_name, network=subnetwork)
#Below is the raw cyREST execution
#requests.get(REST_ENDPOINT + '/v1/apply/layouts/' + layout_name + '/' + subnetwork_suid)

<Response [200]>

In [86]:
#Apply a style
style_name="Minimal"

style = cy.style.create(name=style_name)

cy.style.apply(style, network=subnetwork)
#Below is the raw cyREST execution
#requests.get(REST_ENDPOINT + '/v1/apply/styles/' + style_name + '/' + subnetwork_suid)

<Response [200]>

In [87]:
#Upload to CyNDEx
ndex_message_body = {
    'serverUrl': 'http://dev.ndexbio.org/v2', 
    'username':'bsettle', 
    'password':'ndexTest'
}
ndex_response = requests.post(REST_ENDPOINT + '/cyndex2/v1/networks/' + subnetwork_suid, data=json.dumps(ndex_message_body), headers={'Content-Type': 'application/json'})
print('Uploaded network available at: http://dev.ndexbio.org/#/network/' + str(ndex_response.json()["data"]["uuid"]))

Uploaded network available at: http://dev.ndexbio.org/#/network/2305fa32-3c29-11e8-9218-525400c25d22
